In [1]:
import pandas as pd
import numpy as np

### Обработка данных в более удобный формат

In [2]:
df = pd.read_csv('train.csv')
df.head()

,date,sr,fact,enc_hub
0,2021-12-03,1,2,78
1,2022-06-04,27,79,28
2,2021-03-22,9380,11002,104
3,2021-07-31,294,308,70
4,2022-06-04,2,306,92


In [3]:
df.date = df.date.astype(np.datetime64)
_df = df.sort_values(by='date').set_index(['enc_hub', 'date'])

In [4]:
def prep_data(df, lag_counts:(int,int)):
    """
    Возвращает копию заданного датафрейма с 1)
    про-лаггированными исходными переменными `sr`, `fact`,
    а также целевой `y` равной первой доступной следующей `fact` в данном хабе
    """
    
    df.fact = df.fact.astype(np.float64)
    df.sr = df.sr.astype(np.float64)
    df['y'] = df.groupby(level=0)['fact'].shift(-1) 
    
    for sr_lag in range(1, lag_counts[0]+1):
        df[f'sr_{sr_lag}'] = df.groupby(level=0)['fact'].shift(sr_lag)
    
    for fact_lag in range(1, lag_counts[1]+1):
        df[f'fact_{fact_lag}'] = df.groupby(level=0)['fact'].shift(fact_lag)
    
    return df.dropna().reset_index().sort_values(by='date').reset_index().drop(columns='index')

In [5]:
x = prep_data(_df, [10, 10])
x

,enc_hub,date,sr,fact,y,sr_1,sr_2,sr_3,sr_4,sr_5,...,fact_1,fact_2,fact_3,fact_4,fact_5,fact_6,fact_7,fact_8,fact_9,fact_10
0,128,2021-01-12,1863.0,2646.0,2032.0,2032.0,1723.0,1634.0,1427.0,1607.0,...,2032.0,1723.0,1634.0,1427.0,1607.0,1309.0,2082.0,1625.0,1223.0,106.0
1,72,2021-01-12,2343.0,2343.0,3587.0,2948.0,1504.0,1247.0,2214.0,776.0,...,2948.0,1504.0,1247.0,2214.0,776.0,2336.0,2166.0,2992.0,3610.0,157.0
2,90,2021-01-12,3963.0,4440.0,4497.0,4113.0,3883.0,2464.0,3441.0,2120.0,...,4113.0,3883.0,2464.0,3441.0,2120.0,3523.0,3094.0,2976.0,1433.0,1105.0
3,61,2021-01-12,11598.0,11520.0,11395.0,10667.0,5728.0,6701.0,6639.0,5407.0,...,10667.0,5728.0,6701.0,6639.0,5407.0,7582.0,8917.0,8796.0,3410.0,2260.0
4,50,2021-01-12,242.0,193.0,319.0,201.0,118.0,135.0,132.0,66.0,...,201.0,118.0,135.0,132.0,66.0,122.0,166.0,131.0,66.0,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34952,33,2022-07-15,7125.0,9060.0,5385.0,8715.0,10080.0,8480.0,7966.0,2656.0,...,8715.0,10080.0,8480.0,7966.0,2656.0,6459.0,7711.0,8718.0,8865.0,11098.0
34953,60,2022-07-15,10996.0,13102.0,12023.0,13903.0,14630.0,15849.0,8452.0,5673.0,...,13903.0,14630.0,15849.0,8452.0,5673.0,10838.0,12543.0,12452.0,11073.0,8475.0
34954,47,2022-07-15,18887.0,24037.0,16252.0,24161.0,24738.0,30577.0,21426.0,11582.0,...,24161.0,24738.0,30577.0,21426.0,11582.0,15353.0,23005.0,24878.0,23987.0,25754.0
34955,1,2022-07-15,291.0,327.0,333.0,4.0,9.0,30.0,4.0,647.0,...,4.0,9.0,30.0,4.0,647.0,295.0,412.0,41.0,20.0,354.0


In [6]:
x[x.enc_hub == 63] # пример данных в срезе одного хаба

,enc_hub,date,sr,fact,y,sr_1,sr_2,sr_3,sr_4,sr_5,...,fact_1,fact_2,fact_3,fact_4,fact_5,fact_6,fact_7,fact_8,fact_9,fact_10
8,63,2021-01-12,13076.0,16473.0,15766.0,15783.0,6788.0,12774.0,13817.0,12012.0,...,15783.0,6788.0,12774.0,13817.0,12012.0,13783.0,11648.0,11984.0,11125.0,6200.0
49,63,2021-01-13,16430.0,15766.0,15906.0,16473.0,15783.0,6788.0,12774.0,13817.0,...,16473.0,15783.0,6788.0,12774.0,13817.0,12012.0,13783.0,11648.0,11984.0,11125.0
71,63,2021-01-14,16944.0,15906.0,15615.0,15766.0,16473.0,15783.0,6788.0,12774.0,...,15766.0,16473.0,15783.0,6788.0,12774.0,13817.0,12012.0,13783.0,11648.0,11984.0
124,63,2021-01-15,16387.0,15615.0,15102.0,15906.0,15766.0,16473.0,15783.0,6788.0,...,15906.0,15766.0,16473.0,15783.0,6788.0,12774.0,13817.0,12012.0,13783.0,11648.0
133,63,2021-01-16,15667.0,15102.0,12609.0,15615.0,15906.0,15766.0,16473.0,15783.0,...,15615.0,15906.0,15766.0,16473.0,15783.0,6788.0,12774.0,13817.0,12012.0,13783.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34360,63,2022-07-07,7551.0,4699.0,4708.0,4380.0,13445.0,14030.0,5525.0,11632.0,...,4380.0,13445.0,14030.0,5525.0,11632.0,13654.0,14482.0,17188.0,19669.0,17651.0
34440,63,2022-07-08,4632.0,4708.0,3490.0,4699.0,4380.0,13445.0,14030.0,5525.0,...,4699.0,4380.0,13445.0,14030.0,5525.0,11632.0,13654.0,14482.0,17188.0,19669.0
34501,63,2022-07-09,3167.0,3490.0,1886.0,4708.0,4699.0,4380.0,13445.0,14030.0,...,4708.0,4699.0,4380.0,13445.0,14030.0,5525.0,11632.0,13654.0,14482.0,17188.0
34564,63,2022-07-10,1533.0,1886.0,3788.0,3490.0,4708.0,4699.0,4380.0,13445.0,...,3490.0,4708.0,4699.0,4380.0,13445.0,14030.0,5525.0,11632.0,13654.0,14482.0


### Разбиение на тренировочную и валидационную выборки

In [7]:
train = x[x.date <= '2022-05-01']
val = x[x.date > '2022-05-01']

In [8]:
X_train = train.drop(columns=['date', 'y']) # убираем ненужную дату
y_train = train.y

X_val = val.drop(columns=['date', 'y'])
y_val = val.y

In [9]:
X_train

,enc_hub,sr,fact,sr_1,sr_2,sr_3,sr_4,sr_5,sr_6,sr_7,...,fact_1,fact_2,fact_3,fact_4,fact_5,fact_6,fact_7,fact_8,fact_9,fact_10
0,128,1863.0,2646.0,2032.0,1723.0,1634.0,1427.0,1607.0,1309.0,2082.0,...,2032.0,1723.0,1634.0,1427.0,1607.0,1309.0,2082.0,1625.0,1223.0,106.0
1,72,2343.0,2343.0,2948.0,1504.0,1247.0,2214.0,776.0,2336.0,2166.0,...,2948.0,1504.0,1247.0,2214.0,776.0,2336.0,2166.0,2992.0,3610.0,157.0
2,90,3963.0,4440.0,4113.0,3883.0,2464.0,3441.0,2120.0,3523.0,3094.0,...,4113.0,3883.0,2464.0,3441.0,2120.0,3523.0,3094.0,2976.0,1433.0,1105.0
3,61,11598.0,11520.0,10667.0,5728.0,6701.0,6639.0,5407.0,7582.0,8917.0,...,10667.0,5728.0,6701.0,6639.0,5407.0,7582.0,8917.0,8796.0,3410.0,2260.0
4,50,242.0,193.0,201.0,118.0,135.0,132.0,66.0,122.0,166.0,...,201.0,118.0,135.0,132.0,66.0,122.0,166.0,131.0,66.0,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29513,43,549.0,513.0,901.0,2589.0,5091.0,129.0,4894.0,785.0,14.0,...,901.0,2589.0,5091.0,129.0,4894.0,785.0,14.0,2674.0,1565.0,6896.0
29514,25,512.0,998.0,1371.0,2008.0,1949.0,2151.0,2569.0,2128.0,1345.0,...,1371.0,2008.0,1949.0,2151.0,2569.0,2128.0,1345.0,1550.0,1837.0,2403.0
29515,104,2242.0,3897.0,8317.0,13762.0,11825.0,10293.0,16530.0,10819.0,2912.0,...,8317.0,13762.0,11825.0,10293.0,16530.0,10819.0,2912.0,5926.0,12456.0,13483.0
29516,110,18.0,32.0,354.0,26.0,422.0,40.0,435.0,19.0,173.0,...,354.0,26.0,422.0,40.0,435.0,19.0,173.0,339.0,22.0,337.0


In [10]:
cols = X_train.columns[1:]
cols

Index(['sr', 'fact', 'sr_1', 'sr_2', 'sr_3', 'sr_4', 'sr_5', 'sr_6', 'sr_7',
       'sr_8', 'sr_9', 'sr_10', 'fact_1', 'fact_2', 'fact_3', 'fact_4',
       'fact_5', 'fact_6', 'fact_7', 'fact_8', 'fact_9', 'fact_10'],
      dtype='object')

In [11]:
from sklearn.base import BaseEstimator, TransformerMixin

class IdentityTransformer(BaseEstimator, TransformerMixin):
    """
    Класс-болванчик, нужный для верной обработки номера хаба 
    в препроцессинг-пайплайне
    """
    
    def __init__(self):
        pass
    
    def fit(self, input_array, y=None):
        return self
    
    def transform(self, input_array, y=None):
        return input_array*1
    

### Создание модели на основе градиентного бустинга

In [12]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer, make_column_selector

cols = X_train.columns[1:]

scaler=ColumnTransformer([
    ("num_preprocess", MinMaxScaler(), cols),
    ("dummy", IdentityTransformer(), ['enc_hub'])])
    
gbr = GradientBoostingRegressor(min_samples_leaf=10, n_estimators=1000)

model = Pipeline(steps=[('scaler', scaler), ('regressor', gbr)])

In [13]:
%%time

model.fit(X_train, y_train)

CPU times: user 2min 43s, sys: 57.1 ms, total: 2min 44s
Wall time: 2min 44s


Pipeline(steps=[('scaler',
                 ColumnTransformer(transformers=[('num_preprocess',
                                                  MinMaxScaler(),
                                                  Index(['sr', 'fact', 'sr_1', 'sr_2', 'sr_3', 'sr_4', 'sr_5', 'sr_6', 'sr_7',
       'sr_8', 'sr_9', 'sr_10', 'fact_1', 'fact_2', 'fact_3', 'fact_4',
       'fact_5', 'fact_6', 'fact_7', 'fact_8', 'fact_9', 'fact_10'],
      dtype='object')),
                                                 ('dummy',
                                                  IdentityTransformer(),
                                                  ['enc_hub'])])),
                ('regressor',
                 GradientBoostingRegressor(min_samples_leaf=10,
                                           n_estimators=1000))])

In [14]:
model.score(X_train, y_train)

0.9787933625427572

In [15]:
model.score(X_val, y_val)

0.9040231657169495

In [16]:
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error

def score(X, y, model):
    """
    Возвращает словарь состоящий из 
    номера хаба под вопросом,
    метрики MAE,
    метрики MAPE,
    для данной выборки-модели
    """
    
    indices_dict = X.groupby('enc_hub').indices
    hub_metrics = []
    for key, value in indices_dict.items():
        y_pred = model.predict(X.iloc[value])
        y_true = y.iloc[value]
        mae = mean_absolute_error(y_true, y_pred)
        mape = mean_absolute_percentage_error(y_true, y_pred)
        hub_metrics.append({
            'hub': key,
            'mae': mae,
            'mape': mape
        }
        )
    return hub_metrics

In [26]:
s = score(X_val, y_val, model)
res = pd.DataFrame.from_records(s).sort_values('mape')
res['top, %'] = res.mape.rank(pct = True) * 100 
res

,hub,mae,mape,"top, %"
39,57,1421.013772,0.146627,1.123596
43,62,2286.932679,0.175748,2.247191
15,24,937.057530,0.195462,3.370787
9,12,496.254590,0.202694,4.494382
48,68,3718.861445,0.204228,5.617978
...,...,...,...,...
29,44,990.785101,61.197675,95.505618
54,78,295.360858,69.450739,96.629213
22,36,452.002366,122.579305,97.752809
63,91,1340.756302,133.153310,98.876404


### Сравнение модели с наивным предсказанием равным среднему

In [20]:
interesting_hubs = [57, 62, 24, 36, 91, 66]
r = {}
for hub in interesting_hubs:
    y_true = y_val[X_val['enc_hub']==hub]
    y_pred = [X_train[X_train['enc_hub'] == hub]['fact'].mean()] * len(y_true)
    mape = mean_absolute_percentage_error(
        y_true, 
        y_pred)
    
    r[hub] = mape

In [21]:
r

{57: 0.357055813842447,
 62: 1.2164735188505877,
 24: 0.5355783471959487,
 36: 65.90808376553262,
 91: 243.76908474661445,
 66: 1355.6993466449799}